# Creating a vector store

In [1]:
# path to the created database
VECTOR_DB_PATH="/home/ansary/projects/RAGollama3/data/database"

# The path of the csv file that will be used to create a vectorstore. 
# # The csv will have only one column and each row entry will be treated as a separate document.  
DATA_SOURCE="/home/ansary/projects/RAGollama3/data/source.csv" 

# The model to use in sentence-transformers for creation embedding
EMBEDDING_MODEL="l3cube-pune/bengali-sentence-bert-nli" 

## Vector Store Creation

In [2]:
from sentence_transformers import SentenceTransformer
import torch

# Define the CustomEmbeddings class
class CustomEmbeddings:
    def __init__(self, model_name: str, device: str = None):
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = SentenceTransformer(model_name)
        self.model.to(self.device)

    def embed_documents(self, texts):
        """Embeds a list of documents (texts) into embeddings."""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """Embeds a single query into an embedding."""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


/home/ansary/anaconda3/envs/ragollama/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
#---------------------------------
# imports
#---------------------------------
import os
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from tqdm import tqdm
#--------------------------------------
# global
#--------------------------------------
embedding_model=CustomEmbeddings(EMBEDDING_MODEL)

#--------------------------------------
# helper functions
#--------------------------------------
def create_vector_store(df: pd.DataFrame, 
                        vector_db_path: str) -> None:
    """
    Create a vector store from a dataframe with a single column, where each entry in the dataframe
    is treated as a separate document.

    Args:
        df (pd.DataFrame): DataFrame containing a single column with text data.
        vector_db_path (str): Path to the directory where the vector store will be persisted.
    """
    # Check if GPU is available and set the device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Create data directory structure
    os.makedirs(vector_db_path, exist_ok=True)

    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        length_function=len
    )

    
    # Initialize the vector store with GPU-based embeddings
    vectorstore = Chroma(persist_directory=vector_db_path, embedding_function=embedding_model)

    # Process each document in the dataframe
    for data in tqdm(df.iloc[:, 0]):
        documents = [Document(page_content=data)]
        all_splits = text_splitter.split_documents(documents)

        # Add the split documents to the vector store
        vectorstore.add_documents(all_splits)

    # Persist the vector store
    vectorstore.persist()
    print("Vector store created and persisted at:", vector_db_path)


/home/ansary/anaconda3/envs/ragollama/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
df=pd.read_csv(DATA_SOURCE)
df

,row
0,মো: সাবিরুল ইসলাম ঢাকা বিভাগ এর বিভাগীয় কমিশনা...
1,মোহাম্মদ মমিনুর রহমান ঢাকা বিভাগ এর অতিরিক্ত ব...
2,বিশ্বাস রাসেল হোসেন ঢাকা বিভাগ এর অতিরিক্ত বিভ...
3,মেহেদী হাসান ঢাকা বিভাগ এর উপ-পরিচালক (স্হানীয়...
4,তাহমিনা আক্তার ঢাকা বিভাগ এর সিনিয়র সহকারী কমি...
...,...
217,"আফরোজ আরা পারভীন শিক্ষা প্রকৌশল অধিদপ্তর, ঢাকা..."
218,"মো. শফিকুল ইসলাম শিক্ষা প্রকৌশল অধিদপ্তর, ঢাকা..."
219,"মুহাম্মদ বাবুল আহমেদ শিক্ষা প্রকৌশল অধিদপ্তর, ..."
220,"মো. বেল্লাল হোসেন শিক্ষা প্রকৌশল অধিদপ্তর, ঢাক..."


In [5]:
create_vector_store(df,VECTOR_DB_PATH)

Using device: cuda


/home/ansary/anaconda3/envs/ragollama/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
100%|██████████| 222/222 [00:03<00:00, 64.79it/s]


Vector store created and persisted at: /home/ansary/projects/RAGollama3/data/database


/home/ansary/anaconda3/envs/ragollama/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


# Checking Validity of vector store

In [6]:
from langchain.vectorstores import Chroma
from langchain.schema import Document
import torch

def query_vector_store(vector_db_path: str, query_text: str, top_k: int = 5) -> None:
    """
    Query the vector store to find and display the most similar documents to the given query text.

    Args:
        vector_db_path (str): Path to the directory where the vector store is persisted.
        query_text (str): The text query for which to retrieve similar documents.
        top_k (int): The number of top similar documents to retrieve.
    """
    # Load the vector store
    vectorstore = Chroma(persist_directory=vector_db_path, embedding_function=embedding_model)

    # Check if GPU is available and set the device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Perform the query
    results = vectorstore.similarity_search(query_text, k=top_k)
    
    # Print the results
    for i, result in enumerate(results):
        print(f"Document {i+1}:")
        print(result.page_content)
        #print("Similarity score:", result.score)
        print()



In [7]:
# Example usage
query_vector_store(vector_db_path=VECTOR_DB_PATH, query_text='ঢাকা বিভাগ এর বিভাগীয় কমিশনার')

Using device: cuda
Document 1:
মোঃ ছায়াদ আহমদ ঢাকা বিভাগ এর প্রশাসনিক কর্মকর্তা হিসাবে কর্মরত আছেন। মোঃ ছায়াদ আহমদ এর ই-মেইল এড্রেস : acestdivcomdhaka@mopa.gov.bd, অফিসের ফোন নম্বর : ৮৩৯১৬৪১, বাসার ফোন নম্বর : , মোবাইল : 01797751895 ।

Document 2:
মোঃ ছায়াদ আহমদ ঢাকা বিভাগ এর প্রশাসনিক কর্মকর্তা হিসাবে কর্মরত আছেন। মোঃ ছায়াদ আহমদ এর ই-মেইল এড্রেস : acestdivcomdhaka@mopa.gov.bd, অফিসের ফোন নম্বর : ৮৩৯১৬৪১, বাসার ফোন নম্বর : , মোবাইল : 01797751895 ।

Document 3:
মো: সাবিরুল ইসলাম ঢাকা বিভাগ এর বিভাগীয় কমিশনার হিসাবে কর্মরত আছেন। মো: সাবিরুল ইসলাম এর ই-মেইল এড্রেস : divcomdhaka@mopa.gov.bd, অফিসের ফোন নম্বর : 02-48315085, বাসার ফোন নম্বর : 02-48315084, মোবাইল : Mobile:  01713062404, Tel: 02-48315085 (Office), 02-48315084 (Bungalow)49349999 (Office), 8322255 (Bungalow) ।

Document 4:
মো: সাবিরুল ইসলাম ঢাকা বিভাগ এর বিভাগীয় কমিশনার হিসাবে কর্মরত আছেন। মো: সাবিরুল ইসলাম এর ই-মেইল এড্রেস : divcomdhaka@mopa.gov.bd, অফিসের ফোন নম্বর : 02-48315085, বাসার ফোন নম্বর : 02-48315084, মোবাইল : Mobile: